Using Databricks, here is an example of spotify data extraction.

In [ ]:
# Import packages
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
import pandas as pd
import time
import json

In [ ]:
client_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # Insert client_id and client_secret
client_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
redirect_uri = 'http://localhost:3000'
scope = 'user-library-read'
sp_oauth = SpotifyOAuth(client_id, client_secret, redirect_uri, scope=scope)

auth_url = sp_oauth.get_authorize_url()
print("Visit this URL to authorize your application: " + auth_url)
response = input("Enter the URL you were redirected to: ")

In [ ]:
# Get the authorization code from the URL
code = sp_oauth.parse_response_code(response)
print("Authorization_code: ",code)

In [ ]:
# Spotify API endpoint for token retrieval
token_url = 'https://accounts.spotify.com/api/token'

# Define the payload for token retrieval
data = {
    'code': code,
    'redirect_uri': redirect_uri,
    'grant_type': "authorization_code"
}

In [ ]:
auth_header = {
    'Authorization': 'Basic ' + base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()
}

# Request the access token
response = requests.post(token_url, data=data, headers=auth_header)
token_data = response.json()
print(token_data)

In [ ]:
access_token ='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # Insert access_token

In [ ]:
        def API_search_request(query, results_limit, results_offset, xtrack):

            off = str(results_offset)
            lim = str(results_limit)
            url = "https://api.spotify.com/v1/search"
            headers = {'Authorization': f'Bearer {access_token}'}
            params = {"q": query, "type": "track", "limit": lim, "offset": off}

            response = requests.get(url, headers=headers, params=params)
            df1 = None

            if response.status_code == 200:
                data = response.json()
                items = data.get('tracks', {}).get('items', [])

                for i in items:
                    k = {
                        'popularity': i.get('popularity', None),
                        'song_id': i.get('id', None),
                        'artist_id': i['artists'][0].get('id', None),
                        'album_id': i['album'].get('id', None),
                        'name': i.get('name', None),
                        'artist_name': i['artists'][0].get('name', None),
                        'album_name': i['album'].get('name', None),
                        'explicit': i.get('explicit', False),
                        'disc_number': i.get('disc_number', None),
                        'track_number': i.get('track_number', None)
                    }
                    xtrack.append(k)
                    df1 = pd.DataFrame (xtrack)


            else:
                print(f"Request failed with status code: {response.status_code}")
                print(response.text)

            xaudio=[]

            if df1 is not None:
                song_id = df1['song_id']
                url = 'https://api.spotify.com/v1/audio-features'
                headers = {"Authorization": "Bearer " + access_token, "Accept": "application/json"}

                # Define the batch size (50 in this case)
                batch_size = 50

                # Create a loop to process the track IDs in batches
                for i in range(0, len(song_id), batch_size):
                    batch_ids = song_id[i:i + batch_size]
                    track_ids = ','.join(batch_ids)

                    # Set the parameters for the request
                    params = {'ids': track_ids}

                    # Make the API request
                    track_response = requests.get(url, headers=headers, params=params)

                    if track_response.status_code == 200:
                        audio_data = track_response.json()
                        audio_features = audio_data.get('audio_features', [])

                        for audio_feature in audio_features:
                             if audio_feature is not None:
                                k = {
                                    "acousticness": audio_feature.get('acousticness', None),
                                    "danceability": audio_feature.get('danceability', None),
                                    "duration_ms": audio_feature.get('duration_ms', None),
                                    "song_id": audio_feature.get('id', None),
                                    "instrumentalness": audio_feature.get('instrumentalness', None),
                                    "energy": audio_feature.get('energy', None),
                                    "key": audio_feature.get('key', None),
                                    "liveness": audio_feature.get('liveness', None),
                                    "loudness": audio_feature.get('loudness', None),
                                    "mode": audio_feature.get('mode', None),
                                    "speechiness": audio_feature.get('speechiness', None),
                                    "valence": audio_feature.get('valence', None),
                                    "tempo": audio_feature.get('tempo', None),
                                    "time_signature": audio_feature.get('time_signature', None),
                                    "genres": audio_feature.get('genres', None),
                                    "release_date": audio_feature.get('release_date', None)
                                }
                                xaudio.append(k)
                    else:
                        print(f"Request failed with status code: {track_response.status_code}")
                        print(track_response.text)

                    time.sleep(3)

                df2= pd.DataFrame (xaudio)
                df = df1.merge(df2, on='song_id', how='outer')

                spark_df = spark.createDataFrame(df)
                ## Write Frame out as Table
                spark_df.write.mode("overwrite").saveAsTable("df")

if __name__ == '__main__':
    main()